## **1. Importação de pacotes e funções**

In [0]:

from pyspark.sql.functions import lit, col, sum, when, split, explode, trim, lower
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import Row
import pandas as pd
from scipy.stats import f_oneway


## **2. Importação dos dados** 

In [0]:
# 1. Download dos arquivos para /tmp/
!wget -O /tmp/apple_TV.csv https://raw.githubusercontent.com/pedro1999-wolf/MVP---Engenharia-de-dados/main/apple_TV.csv
!wget -O /tmp/hbo.csv https://raw.githubusercontent.com/pedro1999-wolf/MVP---Engenharia-de-dados/main/hbo.csv
!wget -O /tmp/hulu.csv https://raw.githubusercontent.com/pedro1999-wolf/MVP---Engenharia-de-dados/main/hulu.csv
!wget -O /tmp/netflix.csv https://raw.githubusercontent.com/pedro1999-wolf/MVP---Engenharia-de-dados/main/netflix.csv
!wget -O /tmp/prime_video.csv https://raw.githubusercontent.com/pedro1999-wolf/MVP---Engenharia-de-dados/main/prime_video.csv

# 2. Leitura dos CSVs com Spark
apple_tv_df = spark.read.option("header", "true").csv("file:/tmp/apple_TV.csv").withColumn("plataforma", lit("Apple TV"))
hbo_df = spark.read.option("header", "true").csv("file:/tmp/hbo.csv").withColumn("plataforma", lit("HBO"))
hulu_df = spark.read.option("header", "true").csv("file:/tmp/hulu.csv").withColumn("plataforma", lit("Hulu"))
netflix_df = spark.read.option("header", "true").csv("file:/tmp/netflix.csv").withColumn("plataforma", lit("Netflix"))
prime_video_df = spark.read.option("header", "true").csv("file:/tmp/prime_video.csv").withColumn("plataforma", lit("Prime Video"))

--2025-04-08 14:52:40--  https://raw.githubusercontent.com/pedro1999-wolf/MVP---Engenharia-de-dados/main/apple_TV.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194298 (1.1M) [text/plain]
Saving to: ‘/tmp/apple_TV.csv’

/tmp/apple_TV.csv   100%[===================>]   1.14M  --.-KB/s    in 0.05s   

2025-04-08 14:52:40 (23.1 MB/s) - ‘/tmp/apple_TV.csv’ saved [1194298/1194298]

--2025-04-08 14:52:41--  https://raw.githubusercontent.com/pedro1999-wolf/MVP---Engenharia-de-dados/main/hbo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 

## **3. Processamento dos dados**


### **3.1. União das tabelas em um dataframe**

In [0]:
# 3. União dos dataframes
df_total = apple_tv_df.unionByName(hbo_df, allowMissingColumns=True) \
                      .unionByName(hulu_df, allowMissingColumns=True) \
                      .unionByName(netflix_df, allowMissingColumns=True) \
                      .unionByName(prime_video_df, allowMissingColumns=True)

# 4. Checar se a união deu certo
df_total.show(10)


+--------------------+-----+--------------------+-----------+---------+-----------------+------------+------------------+----------+
|               title| type|              genres|releaseYear|   imdbId|imdbAverageRating|imdbNumVotes|availableCountries|plataforma|
+--------------------+-----+--------------------+-----------+---------+-----------------+------------+------------------+----------+
|          Four Rooms|movie|              Comedy|       1995|tt0113101|              6.7|      113837|              null|  Apple TV|
|        Forrest Gump|movie|      Drama, Romance|       1994|tt0109830|              8.8|     2365858|              null|  Apple TV|
|     American Beauty|movie|               Drama|       1999|tt0169547|              8.3|     1245164|              null|  Apple TV|
|        Citizen Kane|movie|      Drama, Mystery|       1941|tt0033467|              8.3|      479414|              null|  Apple TV|
|          Metropolis|movie|       Drama, Sci-Fi|       1927|tt001713

### **3.2. Explosão da coluna de gêneros**

In [0]:
# Explodir os gêneros separados por vírgula
df_generos_explodido = df_total.withColumn("genres", explode(split(col("genres"), ",")))
df_generos_explodido = df_generos_explodido.withColumn("genres", lower(trim(col("genres"))))

# Checar se a explosão deu certo
df_generos_explodido.show(10)


+---------------+-----+-------+-----------+---------+-----------------+------------+------------------+----------+
|          title| type| genres|releaseYear|   imdbId|imdbAverageRating|imdbNumVotes|availableCountries|plataforma|
+---------------+-----+-------+-----------+---------+-----------------+------------+------------------+----------+
|     Four Rooms|movie| comedy|       1995|tt0113101|              6.7|      113837|              null|  Apple TV|
|   Forrest Gump|movie|  drama|       1994|tt0109830|              8.8|     2365858|              null|  Apple TV|
|   Forrest Gump|movie|romance|       1994|tt0109830|              8.8|     2365858|              null|  Apple TV|
|American Beauty|movie|  drama|       1999|tt0169547|              8.3|     1245164|              null|  Apple TV|
|   Citizen Kane|movie|  drama|       1941|tt0033467|              8.3|      479414|              null|  Apple TV|
|   Citizen Kane|movie|mystery|       1941|tt0033467|              8.3|      479

### **3.3. Limpeza dos dados**

In [0]:
# Verificar número de nulos por coluna
df_generos_explodido.select([
    sum(when(col(c).isNull() | (col(c) == ""), 1).otherwise(0)).alias(c)
    for c in df_generos_explodido.columns
]).show()

# Eliminar colunas não informativas
df_drop_colunas = df_generos_explodido.drop("availableCountries", "imdbNumVotes", "imdbId", "releaseYear")

# Eliminar linhas de valores nulos e produzir a tabela final
df_limpo = df_drop_colunas.na.drop()

# Verificar se as colunas e nulos foram eliminados
df_limpo.select([
    sum(when(col(c).isNull() | (col(c) == ""), 1).otherwise(0)).alias(c)
    for c in df_limpo.columns
]).show()

+-----+----+------+-----------+------+-----------------+------------+------------------+----------+
|title|type|genres|releaseYear|imdbId|imdbAverageRating|imdbNumVotes|availableCountries|plataforma|
+-----+----+------+-----------+------+-----------------+------------+------------------+----------+
| 4160|   0|     0|        153| 11981|            16872|       16872|            260849|         0|
+-----+----+------+-----------+------+-----------------+------------+------------------+----------+

+-----+----+------+-----------------+----------+
|title|type|genres|imdbAverageRating|plataforma|
+-----+----+------+-----------------+----------+
|    0|   0|     0|                0|         0|
+-----+----+------+-----------------+----------+



In [0]:
# Checando se informações por coluna
df_limpo.groupBy("plataforma").count().show()
df_limpo.groupBy("genres").count().show()
df_limpo.groupBy("type").count().show()

+-----------+------+
| plataforma| count|
+-----------+------+
|   Apple TV| 34411|
|        HBO| 18558|
|       Hulu| 20626|
|    Netflix| 43109|
|Prime Video|128330|
+-----------+------+

+-----------+-----+
|     genres|count|
+-----------+-----+
|  biography| 4905|
|      crime|15620|
|    fantasy| 5543|
|documentary|13289|
|       news|  139|
|     action|18805|
|  animation| 8715|
|       soap|    1|
|    mystery| 8640|
|      sport| 2019|
|     family| 6697|
|     horror|11511|
|  film-noir|  264|
|    history| 4232|
|      music| 2860|
|  talk-show|  176|
|    musical| 1340|
|  adventure|12215|
| reality-tv| 2518|
|      drama|53117|
+-----------+-----+
only showing top 20 rows

+-------------+------+
|         type| count|
+-------------+------+
|           tv| 46017|
|        movie|199014|
| "" Princess"|     3|
+-------------+------+



In [0]:
#Eliminar tipo "Princess" por que não faz sentido
df_limpo = df_limpo.filter(trim(col("type")) != '"" Princess"')

#Eliminar gêneros não informativos, deixando apenas os mais gerais
generos_excluir = [
    "tv movie", "adult", "adventure", "biography", "crime", "family",
    "film-noir", "talk-show", "reality-tv", "war", "soap", "mystery",
    "sport", "news", "short", "western", "game-show", "action & adventure", "animation", "documentary", "fantasy", "history", "musical","music", "kids", "reality"
]
df_limpo = df_limpo.filter(~col("genres").isin(generos_excluir))

# Ajustando valores redundantes em gêneros
df_limpo = df_limpo.withColumn(
    "genres",
    when((col("genres") == "sci-fi") | (col("genres") == "science fiction"), "sci fi")
    .otherwise(col("genres"))
)

In [0]:
# Checando se as eliminações deram certo
df_limpo.groupBy("plataforma").count().show()
df_limpo.groupBy("genres").count().show()
df_limpo.groupBy("type").count().show()

+-----------+-----+
| plataforma|count|
+-----------+-----+
|   Apple TV|21449|
|        HBO| 9333|
|       Hulu|11998|
|    Netflix|26574|
|Prime Video|80957|
+-----------+-----+

+--------+-----+
|  genres|count|
+--------+-----+
|  action|18805|
|  sci fi| 4405|
|  horror|11511|
|   drama|53117|
| romance|14941|
|thriller|14690|
|  comedy|32842|
+--------+-----+

+-----+------+
| type| count|
+-----+------+
|   tv| 20792|
|movie|129519|
+-----+------+



### **3.4. Transformação final dos dados**

In [0]:
# Converter para pandas apenas colunas necessárias
df_limpo_pandas = df_limpo.select("type","genres", "plataforma", "imdbAverageRating").toPandas()

# Definir os gêneros e tipos para os testes de anova
generos = ["comedy", "action", "drama", "romance", "sci fi", "horror", "thriller"]
tipos = ["movie", "tv"]

# Inicializar lista de registros para compor a tabela
registros = []

# Função para criação das variáveis estatísticas
for tipo in tipos:
    for genero in generos:
        df_genero = df_limpo_pandas[
            (df_limpo_pandas["type"].str.lower() == tipo.lower()) &
            (df_limpo_pandas["genres"].str.lower() == genero.lower())
        ][["plataforma", "imdbAverageRating"]].copy()

        df_genero["imdbAverageRating"] = pd.to_numeric(df_genero["imdbAverageRating"], errors="coerce")
        df_genero = df_genero.dropna()

        if df_genero.empty or df_genero["plataforma"].nunique() < 2:
            continue

        ranking = df_genero.groupby("plataforma")["imdbAverageRating"].mean().sort_values(ascending=False)
        plataforma_top1 = ranking.index[0]
        notas_top1 = df_genero[df_genero["plataforma"] == plataforma_top1]["imdbAverageRating"]

        for pos, (plataforma, media) in enumerate(ranking.items(), start=1):
            if plataforma == plataforma_top1:
                p_valor = None
                sig_diff = False
            else:
                notas_outro = df_genero[df_genero["plataforma"] == plataforma]["imdbAverageRating"]
                if len(notas_outro) > 1:
                    stat, p_valor = f_oneway(notas_top1, notas_outro)
                    sig_diff = p_valor < 0.05
                else:
                    p_valor = None
                    sig_diff = False

            registros.append({
                "type": tipo.lower(),
                "genero": genero.lower(),
                "plataforma": plataforma,
                "media_imdb": round(media, 2),
                "ranking": pos,
                "p_valor_vs_top1": round(p_valor, 4) if p_valor is not None else None,
                "diferenca_significativa": sig_diff
            })

# Transformar em DataFrame
df_dim_ranking = pd.DataFrame(registros)
df_final = spark.createDataFrame(df_dim_ranking)

# Checando o dataframe
df_final.show()


+-----+-------+-----------+----------+-------+---------------+-----------------------+
| type| genero| plataforma|media_imdb|ranking|p_valor_vs_top1|diferenca_significativa|
+-----+-------+-----------+----------+-------+---------------+-----------------------+
|movie| comedy|        HBO|      6.32|      1|           null|                  false|
|movie| comedy|       Hulu|      6.28|      2|         0.2892|                  false|
|movie| comedy|   Apple TV|      6.12|      3|            0.0|                   true|
|movie| comedy|    Netflix|       6.0|      4|            0.0|                   true|
|movie| comedy|Prime Video|      5.79|      5|            0.0|                   true|
|movie| action|        HBO|      6.35|      1|           null|                  false|
|movie| action|       Hulu|      6.25|      2|         0.0126|                   true|
|movie| action|    Netflix|      6.12|      3|            0.0|                   true|
|movie| action|   Apple TV|      5.79|     

### **3.5. Criação de tabela dimensão de plataformas**

In [0]:

# Criando tabela dimensional de plataformas
plataformas = [
    Row(id_plataforma=1, plataforma="Netflix", preco_mensal_sem_anuncios=59.90),
    Row(id_plataforma=2, plataforma="Prime Video", preco_mensal_sem_anuncios=29.90),
    Row(id_plataforma=3, plataforma="Apple TV", preco_mensal_sem_anuncios=21.90),
    Row(id_plataforma=4, plataforma="HBO", preco_mensal_sem_anuncios=55.90),
    Row(id_plataforma=5, plataforma="Hulu", preco_mensal_sem_anuncios=18.99)
]

# Criando df spark
dim_plataforma = spark.createDataFrame(plataformas)

# Checando
dim_plataforma.show()


+-------------+-----------+-------------------------+
|id_plataforma| plataforma|preco_mensal_sem_anuncios|
+-------------+-----------+-------------------------+
|            1|    Netflix|                     59.9|
|            2|Prime Video|                     29.9|
|            3|   Apple TV|                     21.9|
|            4|        HBO|                     55.9|
|            5|       Hulu|                    18.99|
+-------------+-----------+-------------------------+



### **3.6. Criação da Tabela Fato**

In [0]:
# Substitui 'plataforma' por 'id_plataforma' via JOIN direto
df_final = df_final.join(
    dim_plataforma.select("plataforma", "id_plataforma"),
    on="plataforma",
    how="left"
).drop("plataforma") 

# Checando
df_final.show()

+-----+-------+----------+-------+---------------+-----------------------+-------------+
| type| genero|media_imdb|ranking|p_valor_vs_top1|diferenca_significativa|id_plataforma|
+-----+-------+----------+-------+---------------+-----------------------+-------------+
|movie| comedy|      6.32|      1|           null|                  false|            4|
|movie| action|      6.35|      1|           null|                  false|            4|
|movie| comedy|      5.79|      5|            0.0|                   true|            2|
|movie| comedy|      6.12|      3|            0.0|                   true|            3|
|movie| comedy|       6.0|      4|            0.0|                   true|            1|
|movie| action|      6.12|      3|            0.0|                   true|            1|
|movie| comedy|      6.28|      2|         0.2892|                  false|            5|
|movie| action|      6.25|      2|         0.0126|                   true|            5|
|movie|  drama|      

### **3.7. Salvando as tabelas no Hive**

In [0]:
# Tabela Fato
df_final.write.format("delta").mode("overwrite").saveAsTable("tabela_fato")

# Tabela dimensão de plataformas
dim_plataforma.write.format("delta").mode("overwrite").saveAsTable("dim_plataforma")


# **4. Solução do problema**

### **4.1. Avaliação do catálogo de filmes por gênero em cada plataforma**

#### **Comédia**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'movie'
  AND f.genero = 'comedy'
ORDER BY f.media_imdb DESC

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,6.32,false
Hulu,18.99,6.28,false
Apple TV,21.9,6.12,true
Netflix,59.9,6.0,true
Prime Video,29.9,5.79,true


#### **Ação**


In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'movie'
  AND f.genero = 'action'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,6.35,false
Hulu,18.99,6.25,true
Netflix,59.9,6.12,true
Apple TV,21.9,5.79,true
Prime Video,29.9,5.35,true


#### **Drama**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'movie'
  AND f.genero = 'drama'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,6.7,false
Hulu,18.99,6.47,true
Netflix,59.9,6.4,true
Apple TV,21.9,6.35,true
Prime Video,29.9,6.04,true


#### **Horror**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'movie'
  AND f.genero = 'horror'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,6.01,false
Hulu,18.99,5.54,true
Apple TV,21.9,5.5,true
Netflix,59.9,5.5,true
Prime Video,29.9,4.62,true


#### **Romance**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'movie'
  AND f.genero = 'romance'
ORDER BY f.media_imdb DESC;


plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,6.48,false
Hulu,18.99,6.33,true
Apple TV,21.9,6.28,true
Netflix,59.9,6.16,true
Prime Video,29.9,6.03,true


#### **Suspense**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'movie'
  AND f.genero = 'thriller'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,6.28,false
Netflix,59.9,5.94,true
Hulu,18.99,5.89,true
Apple TV,21.9,5.7,true
Prime Video,29.9,5.23,true


#### **Ficção Científica**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'movie'
  AND f.genero = 'sci fi'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,6.43,false
Netflix,59.9,6.05,true
Hulu,18.99,6.02,true
Apple TV,21.9,5.67,true
Prime Video,29.9,4.89,true


### **4.2. Avaliação do catálogo de séries por gênero em cada plataforma**


#### **Comédia**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'tv'
  AND f.genero = 'comedy'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,7.17,false
Netflix,59.9,7.14,false
Hulu,18.99,7.12,false
Apple TV,21.9,7.05,false
Prime Video,29.9,7.01,true


#### **Ação**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'tv'
  AND f.genero = 'action'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,7.41,false
Netflix,59.9,7.19,true
Hulu,18.99,7.07,true
Prime Video,29.9,6.98,true
Apple TV,21.9,6.97,true


#### **Drama**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'tv'
  AND f.genero = 'drama'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,7.23,false
Hulu,18.99,7.22,false
Netflix,59.9,7.19,false
Prime Video,29.9,7.18,false
Apple TV,21.9,7.16,false


#### **Romance**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'tv'
  AND f.genero = 'romance'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
Netflix,59.9,7.15,false
HBO,55.9,7.04,false
Prime Video,29.9,7.04,true
Hulu,18.99,6.97,true
Apple TV,21.9,6.94,true


#### **Horror**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'tv'
  AND f.genero = 'horror'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
Apple TV,21.9,6.92,false
Hulu,18.99,6.88,false
Prime Video,29.9,6.76,false
HBO,55.9,6.75,false
Netflix,59.9,6.72,false


#### **Suspense**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'tv'
  AND f.genero = 'thriller'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
HBO,55.9,7.16,false
Hulu,18.99,7.13,false
Prime Video,29.9,6.92,true
Netflix,59.9,6.9,true
Apple TV,21.9,6.89,true


#### **Ficção científica**

In [0]:
%sql
SELECT
  p.plataforma,
  p.preco_mensal_sem_anuncios,
  f.media_imdb,
  f.diferenca_significativa
FROM default.tabela_fato f
JOIN default.dim_plataforma p
  ON f.id_plataforma = p.id_plataforma
WHERE f.type = 'tv'
  AND f.genero = 'sci fi'
ORDER BY f.media_imdb DESC;

plataforma,preco_mensal_sem_anuncios,media_imdb,diferenca_significativa
Apple TV,21.9,7.05,false
HBO,55.9,6.78,false
Hulu,18.99,6.67,true
Prime Video,29.9,6.63,true
Netflix,59.9,6.61,true
